In [1]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE

In [2]:
fileloc='D:/data/aman_api_postmatem/data/'

In [3]:
sm = SMOTE(random_state=42)
for i in range(1,29):
    fname=fileloc+str(i)+'.csv'
    df=np.genfromtxt(fname,delimiter=',')
    datan=df[:,0:-1]
    out=df[:,-1]
    X_res, y_res = sm.fit_resample(datan,out)
    y=y_res.reshape(-1,1)
    d=np.concatenate((X_res,y),axis=1)
    fname=fileloc+str(28+i)+".csv"
    np.savetxt(fname,d, delimiter=',', fmt='%f')

In [38]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [40]:
def modeln1(trdata,tract,tsdata):
    model = KNeighborsClassifier(n_neighbors=5)
    model.fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1
def modeln2(trdata,tract,tsdata):
    model = SVC(kernel = 'linear', C = 1,probability=True)
    model.fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1
def modeln3(trdata,tract,tsdata):
    model = GaussianNB()
    model.fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1
def modeln4(trdata,tract,tsdata):
    model = DecisionTreeClassifier().fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1

In [36]:
def normalizedata(X_train):
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    return X_train
def model8(trdata,tract,tsdata):
    model = BaggingClassifier(KNeighborsClassifier(),max_samples=0.5, max_features=0.5).fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1
def model9(trdata,tract,tsdata):
    model = RandomForestClassifier(n_estimators=10).fit(trdata,tract)
    model.fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1
def model10(trdata,tract,tsdata):
    model = ExtraTreesClassifier(n_estimators=10, max_depth=None,min_samples_split=2, random_state=0).fit(trdata,tract)
    model.fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1
def model11(trdata,tract,tsdata):
    model = AdaBoostClassifier(n_estimators=10).fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1
def model12(trdata,tract,tsdata):
    model = GradientBoostingClassifier(n_estimators=10, learning_rate=1.0,max_depth=1, random_state=0).fit(trdata,tract)
    pred= model.predict(tsdata)
    pred1=model.predict_proba(tsdata)
    return pred,pred1

In [41]:
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import KFold
kf = KFold(10)
for i in range(1,57):
    print(i)
    fname=fileloc+str(i)+'.csv'
    data=np.genfromtxt(fname,delimiter=',')
    predvalue=np.zeros((np.shape(data)[0],6))
    predvalue1=np.zeros((np.shape(data)[0],25))
    data[:,0:-1]=normalizedata(data[:,0:-1])
    for train_index, test_index in kf.split(data):
        trdata=data[train_index,0:-1]
        tsdata=data[test_index,0:-1]
        tract=data[train_index,-1]
        tsact=data[test_index,-1]
        predvalue[test_index,0],predvalue1[test_index,0:4]=modeln1(trdata,tract,tsdata)
        predvalue[test_index,1],predvalue1[test_index,4:8]=modeln2(trdata,tract,tsdata)
        predvalue[test_index,2],predvalue1[test_index,8:12]=modeln3(trdata,tract,tsdata)
        predvalue[test_index,3],predvalue1[test_index,12:16]=modeln4(trdata,tract,tsdata)
        predvalue[test_index,4],predvalue1[test_index,16:20]=model9(trdata,tract,tsdata)
        predvalue[test_index,5]=tsact
        predvalue1[test_index,20:24]=to_categorical(tsact,4);
    fname=fileloc+str(i)+'one.csv'    
    np.savetxt(fname,predvalue, delimiter=',', fmt='%f')  
    fname=fileloc+str(i)+'onep.csv'    
    np.savetxt(fname,predvalue1, delimiter=',', fmt='%f')  

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56


In [42]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

In [43]:
rocv=np.zeros((56,5))

for i in range(0,56):
    fname= fileloc+str(i+1)+'onep.csv'
    data=np.genfromtxt(fname,delimiter=',')
    y1=data[:,20:24]
    for j in range(0,5):
        b=data[:,4*j:4*(j+1)]
        r=np.zeros((4))
        for k in range(0,4):
            fpr, tpr, _ = roc_curve(y1[:, k], b[:, k])
            r[k]=auc(fpr, tpr)
        rocv[i,j]=np.mean(r)

In [44]:
from sklearn.metrics import (
    f1_score, precision_score, recall_score,accuracy_score
    )

fval=np.zeros((56,5))
acv=np.zeros((56,5))

   
for i in range(0,56):
    fname= fileloc+str(i+1)+'one.csv'
    data1=np.genfromtxt(fname,delimiter=',')
    y1=data1[:,-1]
    for j in range(0,5):
         fval[i,j]=f1_score(y1, data1[:,j], average="weighted")
         acv[i,j]=accuracy_score(y1, data1[:,j]) 

In [45]:
fname=fileloc+'acc.csv'    
np.savetxt(fname,acv, delimiter=',', fmt='%f')      
fname=fileloc+'fmea.csv'    
np.savetxt(fname,fval, delimiter=',', fmt='%f')  
fname=fileloc+'auc.csv'    
np.savetxt(fname,rocv, delimiter=',', fmt='%f')     

In [31]:
np.shape(rocv)

(56, 5)